1)Написать приложение, которое собирает основные новости с сайтов mail.ru и lenta.ru.

2) (Дополнительно) Собрать все новости со страницы https://yandex.ru/news/

Для парсинга использовать xpath. Структура данных должна содержать:
* название источника,
* наименование новости,
* ссылку на новость,
* дата публикации

In [55]:
import requests

from lxml import html
from pprint import pprint
from pymongo import MongoClient

In [56]:
client = MongoClient('localhost', 27017)
db = client.news_database
collection = db.news

In [57]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/78.0.3904.97 Safari/537.36'}

In [79]:
url_mail = 'https://mail.ru/'

In [58]:
request = requests.get(url_mail, headers=header)

In [59]:
root = html.fromstring(request.text)
result_list = root.xpath('//div[contains(@class, "news-item")]/a[contains(@href, "https")]')

In [60]:
if result_list:
    for i in result_list:
        w = {}
        w['href'] = i.xpath('string(./@href)')
        w['text'] = i.text
        w['date'] = ' '
        w['source'] = 'mail.ru'
        collection.insert_one(w)

In [114]:
def search_news(date, collection):
    for news in collection.find({"date": {"$gt": ''}}):
        print(
            "-" * 79 + f"\n{news['text']}\n{news['href']}\n{news['date']}\n{news['source']}")

In [74]:
url_lenta = 'https://lenta.ru/'

In [75]:
request = requests.get(url_lenta, headers=header)

In [76]:
root = html.fromstring(request.text)
result_list = root.xpath('//section[@class="row b-top7-for-main js-top-seven"]/*/div[@class="item"]/a')

In [77]:
if result_list:
    for i in result_list:
        w = {}
        w['href'] = url_lenta + i.xpath('string(./@href)')
        w['text'] = i.xpath('string(./text())')
        w['date'] = i.xpath('string(./time/@datetime)')
        w['source'] = 'lenta.ru'
        collection.insert_one(w)
#         print(f'{w["href"]}{w["text"]}{w["date"]}{w["source"]}')

In [90]:
url_yandex = 'https://yandex.ru/news'

In [91]:
request = requests.get(url_yandex, headers=header)

In [92]:
root = html.fromstring(request.text)

In [97]:
result_list = root.xpath('//div[contains(@class, "story story_view")]')

In [112]:
if result_list:
    for i in result_list:
        w = {}
        iw = i.xpath('.//a[@class="link link_theme_black i-bem"]')
        w['href'] = url_yandex[:-5] + iw[0].xpath('string(./@href)')
        w['text'] = iw[0].xpath('string(./text())')
        w['date'] = i.xpath('.//div[@class="story__date"]/text()')
        w['source'] = 'yandex.ru'
        collection.insert_one(w)
#         print(f'{w["href"]} . {w["text"]} . {w["date"]} . {w["source"]}')

In [115]:
search_news('', collection)

-------------------------------------------------------------------------------
None
https://news.mail.ru/economics/39609704/?frommail=1
 
mail.ru
-------------------------------------------------------------------------------
В Нижегородской области около 30 школьников отравились газом
https://news.mail.ru/incident/39613950/?frommail=1
 
mail.ru
-------------------------------------------------------------------------------
Россиянам хотят разрешить все способы защиты жизни
https://news.mail.ru/politics/39607685/?frommail=1
 
mail.ru
-------------------------------------------------------------------------------
Синоптики предупредили о грядущей «атмосферной впадине»
https://news.mail.ru/society/39608474/?frommail=1
 
mail.ru
-------------------------------------------------------------------------------
Глава РУСАДА об отстранении от Олимпиады: Это будет трагедия
https://sportmail.ru/news/olympics/39610011/?frommail=10
 
mail.ru
-------------------------------------------------------